In [1]:
import pandas as pd
import numpy as np
import os

### Etapa 1 - Criação do dataframe com dados populacionais por ano dos municípios brasileiros para servir de base de cáclulo das variáveis [QUANTIDADE DE PROCEDIMENTOS POR 10 MIL HAB] E [VALORES DE PROCEDIEMNTOS POR 10 MIL HAB]

In [2]:
# Criando função para concatenar poulação dos municípios por ano, entre 2010 e 2022, a partir dos dados em arquivo csv na pasta ./data/demografico/ para uso em cálculo posterior
def load_population_data(folder_path):
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            year = int(filename.split('.')[0])
            df = pd.read_csv(os.path.join(folder_path, filename), sep=';', names=['codigo', 'pop'], dtype={'codigo': str})
            df.set_index('codigo', inplace=True)
            df = df[~df.index.duplicated(keep='first')] # remove possíveis linhas duplicadas
            df.rename(columns={'pop': str(year)}, inplace=True)
            dfs.append(df.iloc[1:, :]) # ignora a primeira linha
    merged_df = pd.concat(dfs, axis=1)
    merged_df.fillna(0, inplace=True) # preenche valores nulos com 0
    return merged_df


In [3]:
# Aplicando a função fornecendo a pasta com dados como parâmetro
# OBS: esta pasta deve conter somente estes arquivos. Não colocar outros arquivos nesta pasta pois a funçãqo retornará erro!
df_pop = load_population_data('E:/OneDrive/Documentos/GitHub/iris/data/demografico')  

In [4]:
# checando o dataframe e o resultado da função de concatenação de arquivos csv
df_pop

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1100015,24392,24228,24069,25728,25652,25578,25506,25437,23167,22945,22728,22516,22516
1100023,90353,91570,92747,101269,102860,104401,105896,107345,106168,107863,109523,111148,111148
1100031,6313,6221,6132,6495,6424,6355,6289,6224,5438,5312,5188,5067,5067
1100049,78574,78958,79330,85863,86556,87226,87877,88507,84813,85359,85893,86416,86416
1100056,17029,16939,16852,18041,18013,17986,17959,17934,16444,16323,16204,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,12548,12644,12737,13227,13343,13456,13567,13675,13746,13863,13977,14088,14088
5222054,7371,7475,7576,7933,8053,8171,8286,8397,8611,8743,8873,9002,9002
5222203,4735,4846,4954,5246,5371,5495,5615,5731,6026,6171,6312,6451,6451


In [5]:
# Checando quantidade de municípios
len(df_pop)

5570

In [6]:
# checando tipo de variáveis resultante da função 
df_pop.dtypes

2010    object
2011    object
2012    object
2013    object
2014    object
2015    object
2016    object
2017    object
2018    object
2019    object
2020    object
2021    object
2022    object
dtype: object

In [7]:
# convertendo dados para tipo integer
df_pop = df_pop.astype(int)

In [8]:
# Checando valores iguais a zero em cada coluna
num_zeros_por_coluna = (df_pop == 0).sum()
print(num_zeros_por_coluna)
del num_zeros_por_coluna # exclui a var temp, após verificação

2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    0
2019    0
2020    0
2021    0
2022    0
dtype: int64


Nota importante: cinco municípios dos 5.570 existentes atualmente não constavam nas bases de dados do IBGE entre 2010 e 2012, provavelmente foram criados após essa data. Os municípios tem os seguintes códigos: 1504752, 4212650, 4220000, 4314548, 5006275. Os dados inexistenets foram preenchidos com cálculo de população decrescente de 2012, para 2011 e para 2010 com abse na diferença da população anual dos anos subsequentes. Desse modo a matriz não resultará em dados nulos quando for utilizada para cálculos com outras matrizes para obtenção de dados relacionados à população, como é o caso da quantidade de procedimentos por 10 mil habitantes. 


In [9]:
import requests
# obtendo dados dos municípios listados por meio da API do IBGE

def obter_dados_municipios(codigos):
    dados_municipios = []
    for codigo in codigos.split("|"):
        try:
            url = f'https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{codigo}'
            response = requests.get(url)
            if response.status_code == 200:
                dados_municipio = response.json()
                nome_municipio = dados_municipio['nome']
                uf_municipio = dados_municipio['microrregiao']['mesorregiao']['UF']['sigla']
                dados_municipios.append({'Nome': nome_municipio, 'UF': uf_municipio})
        except requests.exceptions.RequestException as e:
            print(f"Erro ao obter dados do município {codigo}: {e}")
    return dados_municipios



In [10]:
# fornece parâmetros com os códigos dos municípios para obtenção do nome e UF
codigos = '1504752|4212650|4220000|4314548|5006275'
municipios = obter_dados_municipios(codigos)
for municipio in municipios:
    print(f"Nome: {municipio['Nome']}, UF: {municipio['UF']}")


Nome: Mojuí dos Campos, UF: PA
Nome: Pescaria Brava, UF: SC
Nome: Balneário Rincão, UF: SC
Nome: Pinto Bandeira, UF: RS
Nome: Paraíso das Águas, UF: MS


3 criados em 2013, sendo que Balneário Rincão e Pinto Bandeira foram emancipados antes de 2010. De toda forma os dados de população para os 5 municípios nos 3 anos foram projetados de maneira a não causar viés nos dados.

In [11]:
# mostando os dados dos municípios e as projeções resultantes para população nos anos de 2010 a 2012. 
df_pop.loc[['1504752', '4212650', '4220000', '4314548', '5006275']]


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1504752,14905,15014,15123,15232,15341,15446,15548,15646,15982,16084,16184,16282,16282
4212650,9465,9539,9613,9687,9761,9835,9908,9980,10022,10091,10159,10225,10225
4220000,11040,11236,11432,11628,11824,12018,12212,12403,12570,12760,12946,13129,13129
4314548,2324,2443,2562,2681,2800,2824,2847,2868,2968,3003,3036,3068,3068
5006275,4627,4732,4837,4942,5047,5150,5251,5350,5455,5555,5654,5751,5751


### Etapa 2 - Criação do dataframe com dados dos procedimentos para calcular matrizes de treinamento da rede neural autoencoder mlp e dos algoritmos de ML supervisionado e não supervisionado

In [12]:
df_sia = pd.read_csv('./data/dfpo1/SIA_procedimentos.csv', sep='|')
df_sia

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,co_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,329,201105,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108006,ACOMPANHAMENTO INTENSIVO DE PACIENTE EM SAUDE ...,166,3004.60
1,329,201102,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,52,772.20
2,329,201111,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,30,445.50
3,329,201209,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,42,623.70
4,329,201104,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,110,1749.00
...,...,...,...,...,...,...,...,...,...,...,...
7477277,9993827,202009,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60480001,"ENOXAPARINA SÓDICA 40 MG/0,4 ML SOLUÇÃO INJETÁ...",30,0.00
7477278,9993827,202110,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477279,9993827,202208,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477280,9993827,202207,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483002,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 100 MG ...,60,207.60


In [13]:
df_sia.rename(columns={'co_procedimento': 'codigo_procedimento'}, inplace=True)

In [14]:
df_sih = pd.read_csv('./data/dfpo1/SIH_procedimentos.csv', sep='|')
df_sih

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,codigo_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303080078,TRATAMENTO DE ESTREPTOCOCCIAS,11,340.62
1,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303090316,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,11,238.70
2,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303020083,TRATAMENTO DE OUTRAS DOENÇAS DO SANGUE E DOS Ó...,24,388.37
3,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,409060135,HISTERECTOMIA TOTAL,12,658.03
4,35,201004,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303070102,TRATAMENTO DE OUTRAS DOENCAS DO APARELHO DIGES...,15,380.58
...,...,...,...,...,...,...,...,...,...,...,...
28657071,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040099,HERNIOPLASTIA INGUINAL (BILATERAL),27,5490.54
28657072,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),51,10845.49
28657073,9994289,202105,POLICLINICA MUNICIPAL ESTEFANIA FARIAS,PREFEITURA MUNICIPAL DE SURUBIM,PE,261450,SURUBIM,303010223,TRATAMENTO DE INFECÇÃO PELO CORONAVIRUS – COVI...,44,33000.00
28657074,9997784,202107,HOSPITAL E MATERNIDADE NAILA GONCALO,MUNICIPIO DE BACABAEIRA,MA,210125,BACABEIRA,411010034,PARTO CESARIANO,74,4867.57


In [15]:
df_sia['fonte'] = 'sia'
df_sih['fonte'] = 'sih'

In [16]:
# Definindo a opção "float_format" para exibir somente 1 dígito após a vírgula
pd.set_option("float_format", "{:.2f}".format)

In [17]:
df_sia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7477282 entries, 0 to 7477281
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 684.6+ MB


In [18]:
df_sih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28657076 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     int64  
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.6+ GB


In [19]:
df_procedimentos = pd.concat([df_sia, df_sih])
df_procedimentos.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36134358 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 3.5+ GB


In [20]:
print(df_procedimentos['fonte'].value_counts() / df_procedimentos['fonte'].count() *100)


sih   79.31
sia   20.69
Name: fonte, dtype: float64


In [21]:
# cria um df com base em uma pivot table para os dados de quantidade de procedimento por cnes, com colunas com dados do município
df_qtde_proc_cnes = (
    df_procedimentos
    .pivot_table(index=['codigo_municipio', 'municipio', 'uf', 'cnes', 'codigo_procedimento', 'procedimento'], columns='competencia', values='qtde_procedimentos')
    .reset_index()
    .sort_index()
)

In [22]:
# Convertendo o índice "date" em uma coluna
df_qtde_proc_cnes = df_qtde_proc_cnes.reset_index()

# Removendo o nome da coluna "variable" da primeira linha
df_qtde_proc_cnes = df_qtde_proc_cnes.rename_axis(None, axis=1)
df_qtde_proc_cnes  = df_qtde_proc_cnes.drop('index', axis =1)

In [23]:
# modifica tipo de variável para codigo do município para inteiro
df_qtde_proc_cnes['codigo_municipio'] = df_qtde_proc_cnes['codigo_municipio'].astype(int)
df_qtde_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,301060010,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.00,55.00,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,301060088,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.00,94.00,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010010,TRATAMENTO DE DENGUE CLÁSSICA,139.00,113.00,30.00,19.00,...,12.00,6.00,28.00,138.00,261.00,189.00,91.00,134.00,183.00,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,...,4.00,22.00,12.00,56.00,157.00,NaN,63.00,249.00,2.00,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010045,TRATAMENTO DE DOENÇAS BACTERIANAS ZOONÓTICAS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377489,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101020,CADEIRA DE RODAS MONOBLOCO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377490,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101021,CADEIRA DE RODAS (ACIMA 90KG),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377491,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101022,CADEIRA DE RODAS MOTORIZADA ADULTO OU INFANTIL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377492,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101024,CADEIRA DE RODAS PARA BANHO COM ENCOSTO RECLIN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_qtde_proc_cnes.to_csv('./data/dfpo1/df_qtde_proc_cnes.csv', sep = '|', encoding = 'UTF-8')

In [25]:
# verifica a média de valores NAN mas colunas...  que se mostra muito alto, pois se temos um df com 1,37 milhões de linhas e a média de linhas com valores NaN é de 1,1 milhões, 
# então precisamos selecionar somente os procedimentos com poucos valores ausentes nas linhas tanto para CNES quanto apra municípios
df_qtde_proc_cnes.isna().sum().mean()

1103507.2654320989

In [26]:
df_qtde_proc_mun = (
    df_procedimentos
    .pivot_table(index=['codigo_municipio', 'municipio', 'uf', 'codigo_procedimento', 'procedimento'], columns='competencia', values='qtde_procedimentos')
    .reset_index()
    .sort_index()
)

In [30]:
# Convertendo o índice "date" em uma coluna
df_qtde_proc_mun = df_qtde_proc_mun.reset_index()

# Removendo o nome da coluna "variable" da primeira linha
df_qtde_proc_mun = df_qtde_proc_mun.rename_axis(None, axis=1)
df_qtde_proc_mun  = df_qtde_proc_mun.drop('index', axis =1)
df_qtde_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001.00,ALTA FLORESTA D'OESTE,RO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,NaN,168.00,168.00,768.00,840.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001.00,ALTA FLORESTA D'OESTE,RO,301060010,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.00,55.00,NaN
2,110001.00,ALTA FLORESTA D'OESTE,RO,301060088,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.00,94.00,NaN
3,110001.00,ALTA FLORESTA D'OESTE,RO,303010010,TRATAMENTO DE DENGUE CLÁSSICA,139.00,113.00,30.00,19.00,4.00,...,12.00,6.00,28.00,138.00,261.00,189.00,91.00,134.00,183.00,NaN
4,110001.00,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,10.00,...,4.00,22.00,12.00,56.00,157.00,NaN,63.00,249.00,2.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932634,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120032,MASTECTOMIA SIMPLES EM ONCOLOGIA,NaN,NaN,47.00,5.00,27.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932635,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120040,RESSECÇÃO DE LESÃO NÃO PALPÁVEL DE MAMA COM MA...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932636,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932637,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,505010097,TRANSPLANTE DE CORNEA,54.00,48.00,72.00,33.00,18.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_qtde_proc_mun.to_csv('./data/dfpo1/df_qtde_proc_mun.csv', sep = '|', encoding = 'UTF-8')

In [32]:
df_qtde_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001.00,ALTA FLORESTA D'OESTE,RO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,NaN,168.00,168.00,768.00,840.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001.00,ALTA FLORESTA D'OESTE,RO,301060010,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.00,55.00,NaN
2,110001.00,ALTA FLORESTA D'OESTE,RO,301060088,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.00,94.00,NaN
3,110001.00,ALTA FLORESTA D'OESTE,RO,303010010,TRATAMENTO DE DENGUE CLÁSSICA,139.00,113.00,30.00,19.00,4.00,...,12.00,6.00,28.00,138.00,261.00,189.00,91.00,134.00,183.00,NaN
4,110001.00,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,10.00,...,4.00,22.00,12.00,56.00,157.00,NaN,63.00,249.00,2.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932634,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120032,MASTECTOMIA SIMPLES EM ONCOLOGIA,NaN,NaN,47.00,5.00,27.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932635,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120040,RESSECÇÃO DE LESÃO NÃO PALPÁVEL DE MAMA COM MA...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932636,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932637,530180.00,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,505010097,TRANSPLANTE DE CORNEA,54.00,48.00,72.00,33.00,18.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_qtde_proc_mun.isna().sum().mean()

728710.354037267

In [34]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_cnes.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
procedimentos_cnes_sem_nan = df_qtde_proc_cnes.dropna(thresh=min_valid_values).reset_index()

# Mantém apenas as colunas 'codigo_procedimento' e 'procedimento' e remove duplicatas
procedimentos_cnes_sem_nan = procedimentos_cnes_sem_nan[['codigo_procedimento', 'procedimento']].drop_duplicates()


In [35]:
procedimentos_cnes_sem_nan

,codigo_procedimento,procedimento
0,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS
1,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS
2,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...
3,303060107,TRATAMENTO DE CRISE HIPERTENSIVA
4,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...
...,...,...
54561,408060638,TRATAMENTO CIRÚRGICO DE LUXAÇÃO / FRATURA-LUXA...
54844,303080086,TRATAMENTO DE FARMACODERMIAS
55480,408030321,"ARTRODESE TORACO-LOMBO-SACRA POSTERIOR, SETE N..."
55482,408050667,TRATAMENTO CIRÚRGICO DE LESÃO AGUDA CAPSULO-LI...


In [36]:
procedimentos_cnes_validos = len(procedimentos_cnes_sem_nan)

In [37]:
procedimentos_cnes_sem_nan.to_excel('./data/dfpo1/procedimentos_cnes_iris.xlsx')

NOTA 2: isso implica que no total, seriam 921 os procedimentos POR CNES com no mínimo 90% de valores válidos (não nulos) nas linhas que poderiam ser analizados pela IRIS sem que seja necesário artifícios de tratamento e substituição de missings por dados sintéticos que causem viéses nos dados (substituição de missings por dados calculados, como média ou mediana. Sendo que, para a solução proposta conforme método de auditoria do MS, a mediana seria mais adequada para não repetir os viéses dos outliers de possíveis fraudes e porque é a medida utilizada nos processos de auditoria como no caso do Maranhão)

In [38]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_mun.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
procedimentos_mun_sem_nan = df_qtde_proc_mun.dropna(thresh=min_valid_values).reset_index()

# Mantém apenas as colunas 'codigo_procedimento' e 'procedimento' e remove duplicatas
procedimentos_mun_sem_nan = procedimentos_mun_sem_nan[['codigo_procedimento', 'procedimento']].drop_duplicates()

procedimentos_mun_sem_nan

,codigo_procedimento,procedimento
0,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS
1,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS
2,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...
3,303060107,TRATAMENTO DE CRISE HIPERTENSIVA
4,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...
...,...,...
48522,409060267,SALPINGOPLASTIA VIDEOLAPAROSCOPICA
48542,412030098,PLEUROSTOMIA
48733,408060409,RETIRADA DE TRAÇÃO TRANS-ESQUELÉTICA
52951,408030321,"ARTRODESE TORACO-LOMBO-SACRA POSTERIOR, SETE N..."


In [39]:
procedimentos_mun_validos = len(procedimentos_mun_sem_nan)
procedimentos_mun_validos

1176

In [40]:
procedimentos_mun_sem_nan.to_excel('./data/dfpo1/procedimentos_mun_iris.xlsx')

### df_proc_cnes é o dataframe com as quantidades de procedimentos POR CNES com menos de 10% de valores NaN nas linhas (ou seja, nos 13 anos ou 156 colunas de competencia mes/ano)
### Este df deve ser a base da qual calculamos todas as demais (df com as distâncias das medianas e assim por diante.. e os mesmos procedimentos devem ser feitos com os df com valores)

In [41]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_cnes.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
df_proc_cnes = df_qtde_proc_cnes.dropna(thresh=min_valid_values).reset_index()

df_proc_cnes = df_proc_cnes.drop('index', axis =1)

df_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,...,4.00,22.00,12.00,56.00,157.00,NaN,63.00,249.00,2.00,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.00,109.00,137.00,89.00,...,16.00,23.00,14.00,70.00,105.00,108.00,6.00,45.00,163.00,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.00,NaN,4.00,7.00,...,20.00,16.00,8.00,76.00,307.00,87.00,120.00,46.00,220.00,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.00,11.00,37.00,4.00,...,14.00,16.00,16.00,11.00,91.00,27.00,50.00,8.00,37.00,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.00,18.00,10.00,12.00,...,10.00,8.00,10.00,3.00,33.00,258.00,4.00,17.00,12.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,16.00,84.00,65.00,72.00,...,63.00,63.00,31.00,63.00,63.00,NaN,NaN,NaN,13.00,38.00
55601,530010,BRASILIA,DF,3077098,70103014,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,14.00,36.00,92.00,30.00,...,27.00,27.00,27.00,27.00,27.00,NaN,NaN,NaN,6.00,19.00
55602,530010,BRASILIA,DF,3206874,50301001,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,NaN,62.00,77.00,80.00,...,62.00,82.00,70.00,99.00,53.00,62.00,29.00,66.00,40.00,NaN
55603,530010,BRASILIA,DF,3206874,50401002,PROCESSAMENTO DE CORNEA / ESCLERA,NaN,129.00,170.00,198.00,...,158.00,155.00,165.00,227.00,119.00,142.00,42.00,150.00,88.00,NaN


### df_proc_mun é o dataframe com as quantidades de procedimentos POR MUNICÍPIO com menos de 10% de valores NaN nas linhas (ou seja, nos 13 anos ou 156 colunas de competencia mes/ano)
### Este df deve ser a base da qual calculamos todas as demais (df com as distâncias das medianas, df com a quantidade de procedimentos para cada 10 mil hab. e assim por diante.. e os mesmos procedimentos devem ser feitos com os df com valores)

In [42]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_mun.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
df_proc_mun = df_qtde_proc_mun.dropna(thresh=min_valid_values).reset_index()

# drop coluna desnecassária criada nos procedimetnos anteriores
df_proc_mun = df_proc_mun.drop('index', axis =1)

# modifica tipo de variável para codigo do município para inteiro
df_proc_mun['codigo_municipio'] = df_proc_mun['codigo_municipio'].astype(int)

df_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,10.00,...,4.00,22.00,12.00,56.00,157.00,NaN,63.00,249.00,2.00,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.00,109.00,137.00,89.00,74.00,...,16.00,23.00,14.00,70.00,105.00,108.00,6.00,45.00,163.00,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.00,NaN,4.00,7.00,9.00,...,20.00,16.00,8.00,76.00,307.00,87.00,120.00,46.00,220.00,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.00,11.00,37.00,4.00,19.00,...,14.00,16.00,16.00,11.00,91.00,27.00,50.00,8.00,37.00,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.00,18.00,10.00,12.00,8.00,...,10.00,8.00,10.00,3.00,33.00,258.00,4.00,17.00,12.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,87.50,29.00,16.00,46.33,49.50,...,34.00,43.25,19.80,17.80,27.50,16.00,27.80,123.33,35.00,47.80
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,6.00,NaN,56.50,4.00,23.00,...,76.60,34.00,55.50,40.40,29.83,64.60,48.80,123.00,59.50,47.67
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,16.00,184.00,13.00,26.00,57.00,...,78.00,69.00,27.25,117.67,53.00,149.50,70.67,52.00,81.00,26.67
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,31.00,9.00,47.50,20.50,NaN,...,67.50,116.00,59.50,65.50,55.00,29.00,39.00,84.50,145.50,5.00


In [43]:
# Selecionar apenas colunas numéricas após a quarta coluna
num_cols = df_proc_mun.columns[5:]

# Substituindo NaN pela mediana de cada linha
df_proc_mun[num_cols] = df_proc_mun[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)

# Calculando o desvio da mediana para cada célula
desvios_med_n_mun = df_proc_mun[num_cols].apply(lambda x: x - x.median(), axis=1)

# Criando um novo DataFrame com os desvios_med_n como uma matriz (m)
m_desvios_med_n_mun = pd.DataFrame(desvios_med_n_mun.values, index=df_proc_mun.index, columns=num_cols)

# Concatenando as quatro primeiras colunas do DataFrame original com o novo DataFrame de desvios_med_n_mun
df_desv_med_n_proc_mun = pd.concat([df_proc_mun.iloc[:, :5], m_desvios_med_n_mun], axis=1)
df_desv_med_n_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,-4.00,1.00,4.00,-2.00,2.00,...,-4.00,14.00,4.00,48.00,149.00,0.00,55.00,241.00,-6.00,0.00
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,79.00,72.00,100.00,52.00,37.00,...,-21.00,-14.00,-23.00,33.00,68.00,71.00,-31.00,8.00,126.00,0.00
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,-5.00,0.00,-13.00,-10.00,-8.00,...,3.00,-1.00,-9.00,59.00,290.00,70.00,103.00,29.00,203.00,0.00
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,-11.00,-9.00,17.00,-16.00,-1.00,...,-6.00,-4.00,-4.00,-9.00,71.00,7.00,30.00,-12.00,17.00,0.00
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,-4.00,4.00,-4.00,-2.00,-6.00,...,-4.00,-6.00,-4.00,-11.00,19.00,244.00,-10.00,3.00,-2.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,37.50,-21.00,-34.00,-3.67,-0.50,...,-16.00,-6.75,-30.20,-32.20,-22.50,-34.00,-22.20,73.33,-15.00,-2.20
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,-29.00,0.00,21.50,-31.00,-12.00,...,41.60,-1.00,20.50,5.40,-5.17,29.60,13.80,88.00,24.50,12.67
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,-86.83,81.17,-89.83,-76.83,-45.83,...,-24.83,-33.83,-75.58,14.83,-49.83,46.67,-32.17,-50.83,-21.83,-76.17
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,-20.00,-42.00,-3.50,-30.50,0.00,...,16.50,65.00,8.50,14.50,4.00,-22.00,-12.00,33.50,94.50,-46.00


In [44]:
df_desv_med_n_proc_mun.to_excel('./data/dfpo1/df_desv_med_n_proc_mun.xlsx')

In [45]:
df_proc_mun[num_cols] = df_proc_mun[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)
df_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,10.00,...,4.00,22.00,12.00,56.00,157.00,8.00,63.00,249.00,2.00,8.00
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.00,109.00,137.00,89.00,74.00,...,16.00,23.00,14.00,70.00,105.00,108.00,6.00,45.00,163.00,37.00
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.00,17.00,4.00,7.00,9.00,...,20.00,16.00,8.00,76.00,307.00,87.00,120.00,46.00,220.00,17.00
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.00,11.00,37.00,4.00,19.00,...,14.00,16.00,16.00,11.00,91.00,27.00,50.00,8.00,37.00,20.00
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.00,18.00,10.00,12.00,8.00,...,10.00,8.00,10.00,3.00,33.00,258.00,4.00,17.00,12.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,87.50,29.00,16.00,46.33,49.50,...,34.00,43.25,19.80,17.80,27.50,16.00,27.80,123.33,35.00,47.80
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,6.00,35.00,56.50,4.00,23.00,...,76.60,34.00,55.50,40.40,29.83,64.60,48.80,123.00,59.50,47.67
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,16.00,184.00,13.00,26.00,57.00,...,78.00,69.00,27.25,117.67,53.00,149.50,70.67,52.00,81.00,26.67
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,31.00,9.00,47.50,20.50,51.00,...,67.50,116.00,59.50,65.50,55.00,29.00,39.00,84.50,145.50,5.00


In [46]:
df_proc_mun.to_excel('./data/dfpo1/df_proc_mun.xlsx')

In [47]:
# Selecionar apenas colunas numéricas após a quarta coluna
num_cols = df_proc_cnes.columns[6:]

# Substituindo NaN pela mediana de cada linha
df_proc_cnes[num_cols] = df_proc_cnes[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)

# Calculando o desvio da mediana para cada célula
desvios_med_n_cnes = df_proc_cnes[num_cols].apply(lambda x: x - x.median(), axis=1)

# Criando um novo DataFrame com os desvios_med_n como uma matriz (m)
m_desvios_med_n_cnes = pd.DataFrame(desvios_med_n_cnes.values, index=df_proc_cnes.index, columns=num_cols)

# Concatenando as quatro primeiras colunas do DataFrame original com o novo DataFrame de desvios_med_n_cnes
df_desv_med_n_proc_cnes = pd.concat([df_proc_cnes.iloc[:, :5], m_desvios_med_n_cnes], axis=1)
df_desv_med_n_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,-4.00,1.00,4.00,-2.00,2.00,...,-4.00,14.00,4.00,48.00,149.00,0.00,55.00,241.00,-6.00,0.00
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,79.00,72.00,100.00,52.00,37.00,...,-21.00,-14.00,-23.00,33.00,68.00,71.00,-31.00,8.00,126.00,0.00
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,-5.00,0.00,-13.00,-10.00,-8.00,...,3.00,-1.00,-9.00,59.00,290.00,70.00,103.00,29.00,203.00,0.00
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,-11.00,-9.00,17.00,-16.00,-1.00,...,-6.00,-4.00,-4.00,-9.00,71.00,7.00,30.00,-12.00,17.00,0.00
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,-4.00,4.00,-4.00,-2.00,-6.00,...,-4.00,-6.00,-4.00,-11.00,19.00,244.00,-10.00,3.00,-2.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,-47.00,21.00,2.00,9.00,28.00,...,0.00,0.00,-32.00,0.00,0.00,0.00,0.00,0.00,-50.00,-25.00
55601,530010,BRASILIA,DF,3077098,70103014,-13.00,9.00,65.00,3.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-21.00,-8.00
55602,530010,BRASILIA,DF,3206874,50301001,0.00,-15.00,0.00,3.00,0.00,...,-15.00,5.00,-7.00,22.00,-24.00,-15.00,-48.00,-11.00,-37.00,0.00
55603,530010,BRASILIA,DF,3206874,50401002,0.00,-25.00,16.00,44.00,36.00,...,4.00,1.00,11.00,73.00,-35.00,-12.00,-112.00,-4.00,-66.00,0.00


In [48]:
df_desv_med_n_proc_cnes.to_excel('./data/dfpo1/df_desv_med_n_proc_cnes.xlsx')

In [49]:
df_proc_cnes[num_cols] = df_proc_cnes[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)
df_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,...,4.00,22.00,12.00,56.00,157.00,8.00,63.00,249.00,2.00,8.00
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.00,109.00,137.00,89.00,...,16.00,23.00,14.00,70.00,105.00,108.00,6.00,45.00,163.00,37.00
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.00,17.00,4.00,7.00,...,20.00,16.00,8.00,76.00,307.00,87.00,120.00,46.00,220.00,17.00
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.00,11.00,37.00,4.00,...,14.00,16.00,16.00,11.00,91.00,27.00,50.00,8.00,37.00,20.00
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.00,18.00,10.00,12.00,...,10.00,8.00,10.00,3.00,33.00,258.00,4.00,17.00,12.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,16.00,84.00,65.00,72.00,...,63.00,63.00,31.00,63.00,63.00,63.00,63.00,63.00,13.00,38.00
55601,530010,BRASILIA,DF,3077098,70103014,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,14.00,36.00,92.00,30.00,...,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,6.00,19.00
55602,530010,BRASILIA,DF,3206874,50301001,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,77.00,62.00,77.00,80.00,...,62.00,82.00,70.00,99.00,53.00,62.00,29.00,66.00,40.00,77.00
55603,530010,BRASILIA,DF,3206874,50401002,PROCESSAMENTO DE CORNEA / ESCLERA,154.00,129.00,170.00,198.00,...,158.00,155.00,165.00,227.00,119.00,142.00,42.00,150.00,88.00,154.00


In [50]:
df_proc_cnes.to_excel('./data/dfpo1/df_proc_cnes.xlsx')

### Calculando o df_proc_pop com os dados de quantidade de procedimentos para cada 10 mil habitantes para os 13 anos. 

In [51]:
df_proc_mun.to_excel('./data/dfpo1/df_proc_mun.xlsx')

In [52]:
df_pop

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1100015,24392,24228,24069,25728,25652,25578,25506,25437,23167,22945,22728,22516,22516
1100023,90353,91570,92747,101269,102860,104401,105896,107345,106168,107863,109523,111148,111148
1100031,6313,6221,6132,6495,6424,6355,6289,6224,5438,5312,5188,5067,5067
1100049,78574,78958,79330,85863,86556,87226,87877,88507,84813,85359,85893,86416,86416
1100056,17029,16939,16852,18041,18013,17986,17959,17934,16444,16323,16204,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,12548,12644,12737,13227,13343,13456,13567,13675,13746,13863,13977,14088,14088
5222054,7371,7475,7576,7933,8053,8171,8286,8397,8611,8743,8873,9002,9002
5222203,4735,4846,4954,5246,5371,5495,5615,5731,6026,6171,6312,6451,6451


Os dataframes serão salvos em formato pickle para que estejam disponíveis em outros notebooks

In [53]:
# salvar os dataframes em um arquivo pickle

### POR CNES
# df procedimentos por cnes
df_proc_cnes.to_pickle('./data/dfpo1/df_proc_cnes.pkl')
# matriz de desvios médios por cnes
m_desvios_med_n_cnes.to_pickle('./data/dfpo1/m_desvios_med_n_cnes.pkl')

### POR MUNICÍPIO
# df procedimentos por município
df_proc_mun.to_pickle('./data/dfpo1/df_proc_mun.pkl')
# df populações dos municípios
df_pop.to_pickle('./data/dfpo1/df_pop.pkl')
# matriz de desvios médios por cnes
m_desvios_med_n_mun.to_pickle('./data/dfpo1/m_desvios_med_n_mun.pkl')

In [120]:
# Resetar o índice do df_pop para tornar "codigo" uma coluna
df_pop_reset = df_pop.reset_index()

# Agora, podemos seguir o mesmo processo de antes
df_pop_melt = df_pop_reset.melt(id_vars='codigo', var_name='ano', value_name='pop')

# Repetir cada valor de população 12 vezes (para cada mês)
df_pop_melt = df_pop_melt.loc[df_pop_melt.index.repeat(12)].reset_index(drop=True)

# Criar uma nova coluna para o mês e preenchê-la com valores de 1 a 12 repetidos para cada ano
df_pop_melt['mes'] = np.tile(np.arange(1, 13), len(df_pop_melt) // 12)

# Criar uma nova coluna 'anomes' combinando o ano e o mês
df_pop_melt['anomes'] = df_pop_melt['ano'].astype(str) + df_pop_melt['mes'].apply(lambda x: f'{x:02d}')

# Ajustar o df_pop_melt para ter o mesmo formato que df_proc_municipio
df_pop_reshaped = df_pop_melt.pivot(index='codigo', columns='anomes', values='pop')

# Converter o índice e as colunas de volta para inteiros
df_pop_reshaped.index = df_pop_reshaped.index.astype(int)
df_pop_reshaped.columns = df_pop_reshaped.columns.astype(int)

In [121]:
df_pop_reshaped

anomes,201001,201002,201003,201004,201005,201006,201007,201008,201009,201010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
codigo,,,,,,,,,,,,,,,,,,,,,
1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451


In [122]:
# Se 'codigo' é um nível de índice e não uma coluna, você pode usar reset_index para torná-lo uma coluna novamente.
df_pop_reshaped.reset_index(level=0, inplace=True)

# Como resultado, agora você deve ter 'codigo' como uma coluna normal.


In [123]:
df_pop_reshaped

anomes,codigo,201001,201002,201003,201004,201005,201006,201007,201008,201009,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
1,1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
2,1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
3,1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
4,1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
5566,5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
5567,5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451
5568,5222302,5145,5145,5145,5145,5145,5145,5145,5145,5145,...,5941,5941,5941,5941,5941,5941,5941,5941,5941,5941


In [ ]:
df_proc_mun = df_proc_mun.drop('index', axis=1)

In [133]:
df_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.00,9.00,12.00,6.00,10.00,...,4.00,22.00,12.00,56.00,157.00,8.00,63.00,249.00,2.00,8.00
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.00,109.00,137.00,89.00,74.00,...,16.00,23.00,14.00,70.00,105.00,108.00,6.00,45.00,163.00,37.00
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.00,17.00,4.00,7.00,9.00,...,20.00,16.00,8.00,76.00,307.00,87.00,120.00,46.00,220.00,17.00
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.00,11.00,37.00,4.00,19.00,...,14.00,16.00,16.00,11.00,91.00,27.00,50.00,8.00,37.00,20.00
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.00,18.00,10.00,12.00,8.00,...,10.00,8.00,10.00,3.00,33.00,258.00,4.00,17.00,12.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,87.50,29.00,16.00,46.33,49.50,...,34.00,43.25,19.80,17.80,27.50,16.00,27.80,123.33,35.00,47.80
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,6.00,35.00,56.50,4.00,23.00,...,76.60,34.00,55.50,40.40,29.83,64.60,48.80,123.00,59.50,47.67
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,16.00,184.00,13.00,26.00,57.00,...,78.00,69.00,27.25,117.67,53.00,149.50,70.67,52.00,81.00,26.67
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,31.00,9.00,47.50,20.50,51.00,...,67.50,116.00,59.50,65.50,55.00,29.00,39.00,84.50,145.50,5.00


In [134]:
df_proc_mun.dtypes

codigo_municipio         int64
municipio               object
uf                      object
codigo_procedimento      int64
procedimento            object
                        ...   
202208                 float64
202209                 float64
202210                 float64
202211                 float64
202212                 float64
Length: 161, dtype: object

In [136]:
df_pop_reshaped.dtypes

anomes
codigo    int64
201001    int32
201002    int32
201003    int32
201004    int32
          ...  
202208    int32
202209    int32
202210    int32
202211    int32
202212    int32
Length: 157, dtype: object

In [137]:
df_pop_reshaped

anomes,codigo,201001,201002,201003,201004,201005,201006,201007,201008,201009,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
1,1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
2,1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
3,1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
4,1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
5566,5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
5567,5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451
5568,5222302,5145,5145,5145,5145,5145,5145,5145,5145,5145,...,5941,5941,5941,5941,5941,5941,5941,5941,5941,5941


In [157]:
df_proc_mun.codigo_municipio.unique()

array([110001, 110002, 110003, ..., 522185, 522200, 530010], dtype=int64)

In [160]:
df_pop_reshaped

anomes,codigo,201001,201002,201003,201004,201005,201006,201007,201008,201009,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
1,1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
2,1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
3,1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
4,1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
5566,5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
5567,5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451
5568,5222302,5145,5145,5145,5145,5145,5145,5145,5145,5145,...,5941,5941,5941,5941,5941,5941,5941,5941,5941,5941


In [163]:
df_pop_reshaped.dtypes

anomes
codigo    int64
201001    int32
201002    int32
201003    int32
201004    int32
          ...  
202208    int32
202209    int32
202210    int32
202211    int32
202212    int32
Length: 157, dtype: object

In [166]:
df_pop_reshaped['codigo'].astype(str).str[:-1].astype(int)

0       110001
1       110002
2       110003
3       110004
4       110005
         ...  
5565    522200
5566    522205
5567    522220
5568    522230
5569    530010
Name: codigo, Length: 5570, dtype: int32

In [169]:
# Criar uma cópia de df_pop_reshaped e adicionar uma nova coluna com o código de município ajustado
df_pop_reshaped['codigo_municipio'] = df_pop_reshaped['codigo'].astype(str).str[:-1].astype(int)

In [170]:
df_pop_reshaped

anomes,codigo,201001,201002,201003,201004,201005,201006,201007,201008,201009,...,202204,202205,202206,202207,202208,202209,202210,202211,202212,codigo_municipio
0,1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,110001
1,1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,110002
2,1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,110003
3,1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,110004
4,1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,110005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,522200
5566,5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,522205
5567,5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,522220
5568,5222302,5145,5145,5145,5145,5145,5145,5145,5145,5145,...,5941,5941,5941,5941,5941,5941,5941,5941,5941,522230


In [174]:
df_pop_reshaped

anomes,codigo,201001,201002,201003,201004,201005,201006,201007,201008,201009,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
codigo_municipio,,,,,,,,,,,,,,,,,,,,,
110001,1100015,24392,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
110002,1100023,90353,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
110003,1100031,6313,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
110004,1100049,78574,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
110005,1100056,17029,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522200,5222005,12548,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
522205,5222054,7371,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
522220,5222203,4735,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451


In [178]:
df_pop_reshaped.reset_index(level=0, inplace=True)

In [179]:
df_pop_reshaped

anomes,codigo_municipio,codigo,201001,201002,201003,201004,201005,201006,201007,201008,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,1100015,24392,24392,24392,24392,24392,24392,24392,24392,...,22516,22516,22516,22516,22516,22516,22516,22516,22516,22516
1,110002,1100023,90353,90353,90353,90353,90353,90353,90353,90353,...,111148,111148,111148,111148,111148,111148,111148,111148,111148,111148
2,110003,1100031,6313,6313,6313,6313,6313,6313,6313,6313,...,5067,5067,5067,5067,5067,5067,5067,5067,5067,5067
3,110004,1100049,78574,78574,78574,78574,78574,78574,78574,78574,...,86416,86416,86416,86416,86416,86416,86416,86416,86416,86416
4,110005,1100056,17029,17029,17029,17029,17029,17029,17029,17029,...,16088,16088,16088,16088,16088,16088,16088,16088,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522200,5222005,12548,12548,12548,12548,12548,12548,12548,12548,...,14088,14088,14088,14088,14088,14088,14088,14088,14088,14088
5566,522205,5222054,7371,7371,7371,7371,7371,7371,7371,7371,...,9002,9002,9002,9002,9002,9002,9002,9002,9002,9002
5567,522220,5222203,4735,4735,4735,4735,4735,4735,4735,4735,...,6451,6451,6451,6451,6451,6451,6451,6451,6451,6451
5568,522230,5222302,5145,5145,5145,5145,5145,5145,5145,5145,...,5941,5941,5941,5941,5941,5941,5941,5941,5941,5941


In [182]:
# Criar uma cópia de df_pop_reshaped e adicionar uma nova coluna com o código de município ajustado
df_pop_reshaped['codigo_municipio'] = df_pop_reshaped['codigo'].astype(str).str[:-1].astype(int)

# Definir a nova coluna "codigo_municipio" como índice do DataFrame
df_pop_reshaped.set_index('codigo_municipio', inplace=True)

# Redefinir o índice do DataFrame df_proc_mun
df_proc_mun.reset_index(drop=True, inplace=True)

# Criar uma cópia de df_proc_mun
df_proc_mun_por_10k = df_proc_mun.copy()

# Selecionar as colunas de população em df_pop_reshaped
pop_columns = df_pop_reshaped.columns[1:]

# Mapear as linhas e colunas corretamente
for i in range(len(df_proc_mun)):
    codigo_municipio = df_proc_mun.loc[i, 'codigo_municipio']
    df_proc_mun_por_10k.iloc[i, 5:] = (df_proc_mun.iloc[i, 5:] / df_pop_reshaped.loc[codigo_municipio, pop_columns]) * 10000

# Verificar se há valores NaN após o cálculo
nan_count = df_proc_mun_por_10k.isna().sum().sum()
print("Quantidade de NaN após o cálculo:", nan_count)


Quantidade de NaN após o cálculo: 0


In [183]:
pop_columns

Index([201001, 201002, 201003, 201004, 201005, 201006, 201007, 201008, 201009,
       201010,
       ...
       202203, 202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211,
       202212],
      dtype='object', name='anomes', length=156)

In [184]:
df_proc_mun_por_10k

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,1.64,3.69,4.92,2.46,4.10,...,1.78,9.77,5.33,24.87,69.73,3.55,27.98,110.59,0.89,3.55
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,47.56,44.69,56.17,36.49,30.34,...,7.11,10.21,6.22,31.09,46.63,47.97,2.66,19.99,72.39,16.43
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,4.92,6.97,1.64,2.87,3.69,...,8.88,7.11,3.55,33.75,136.35,38.64,53.30,20.43,97.71,7.55
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,3.69,4.51,15.17,1.64,7.79,...,6.22,7.11,7.11,4.89,40.42,11.99,22.21,3.55,16.43,8.88
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,4.10,7.38,4.10,4.92,3.28,...,4.44,3.55,4.44,1.33,14.66,114.59,1.78,7.55,5.33,6.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,0.34,0.11,0.06,0.18,0.19,...,0.11,0.14,0.06,0.06,0.09,0.05,0.09,0.40,0.11,0.15
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,0.02,0.14,0.22,0.02,0.09,...,0.25,0.11,0.18,0.13,0.10,0.21,0.16,0.40,0.19,0.15
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,0.06,0.72,0.05,0.10,0.22,...,0.25,0.22,0.09,0.38,0.17,0.48,0.23,0.17,0.26,0.09
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,0.12,0.04,0.18,0.08,0.20,...,0.22,0.37,0.19,0.21,0.18,0.09,0.13,0.27,0.47,0.02


In [185]:
# Verificar quantidade de NaN por procedimento por município em df_proc_mun_por_10k
nan_counts = df_proc_mun_por_10k.iloc[:, 5:].isna().sum()

# Verificar quantidade de NaN em df_proc_mun
nan_counts_original = df_proc_mun.iloc[:, 5:].isna().sum()

# Comparar os resultados
comparison = pd.concat([nan_counts, nan_counts_original], axis=1)
comparison.columns = ['NaN em df_proc_mun_por_10k', 'NaN em df_proc_mun']
print(comparison)


        NaN em df_proc_mun_por_10k  NaN em df_proc_mun
201001                           0                   0
201002                           0                   0
201003                           0                   0
201004                           0                   0
201005                           0                   0
...                            ...                 ...
202208                           0                   0
202209                           0                   0
202210                           0                   0
202211                           0                   0
202212                           0                   0

[156 rows x 2 columns]


In [ ]:
### POR MUNICÍPIO
# df quantidade de procedimentos para cada 10 mil habintantes do município
df_proc_mun_por_10k.to_pickle('./data/dfpo1/df_proc_mun_por_10k.pkl')